# Transfer Learning- Resnet 3 : 加入各种特征

参考：
+ [Basic ResNet50 with composites 0.2502LB](https://www.kaggle.com/jasonbenner/basic-resnet50-with-composites-0-2502lb)
+ 另外参考了 Transfer Learning VGG16

In [1]:
#Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import uniform_filter
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

data_dir = '../ShipIceberg/Data/'

def load_data(data_dir):
    train = pd.read_json(data_dir + 'train.json')
#     test = pd.read_json(data_dir + 'test.json')
    import json
    with open('../ShipIceberg/Data/test.json', 'r') as f:
        test = json.load(f)
        test=pd.DataFrame(test)
    
    print('done!')
    '''
    #Fill 'na' angles with mode???
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    '''
    return train, test

train, test = load_data(data_dir)

done!


In [3]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

y_train=train['is_iceberg']

In [4]:
def color_composite(data):
    import cv2
    w,h = 197,197
    rgb_arrays = []
#     !!! 
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
#         band_3 = band_1 / band_2
        band_3 = np.zeros((75,75))
        
        subt = abs(band_1-band_2)
        W1 = subt/subt.max()
        W2=1-W1
        band_3=W1 * band_1 + W2 * band_2

        '''
        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))
        '''

#         rgb = np.dstack((r, g, b))
        rgb = np.dstack((band_1, band_2, band_3))
        #Add in to resize for resnet50 use 197 x 197
        rgb = cv2.resize(rgb, (w,h)).astype(np.float32)
        rgb_arrays.append(rgb)
    return np.array(rgb_arrays)

X_train = color_composite(train)
X_test = color_composite(test)

In [5]:
# 如何改成新的数据
def get_stats(data,label=1):
    data['max'+str(label)] = [np.max(np.array(x)) for x in data['band_'+str(label)] ]
    data['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in data['band_'+str(label)] ]
    data['min'+str(label)] = [np.min(np.array(x)) for x in data['band_'+str(label)] ]
    data['minpos'+str(label)] = [np.argmin(np.array(x)) for x in data['band_'+str(label)] ]
    data['med'+str(label)] = [np.median(np.array(x)) for x in data['band_'+str(label)] ]
    data['std'+str(label)] = [np.std(np.array(x)) for x in data['band_'+str(label)] ]
    data['mean'+str(label)] = [np.mean(np.array(x)) for x in data['band_'+str(label)] ]
    data['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in data['band_'+str(label)] ]
    data['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in data['band_'+str(label)] ]
    data['mid50_'+str(label)] = data['p75_'+str(label)]-data['p25_'+str(label)]

    return data

train = get_stats(train,1)
train = get_stats(train,2)
X_min1 = train['min1']
X_max1 = train['max1']
X_std1 = train['std1']
X_med1 = train['med1']
X_mean1 = train['mean1']
X_min2 = train['min2']
X_max2 = train['max2']
X_std2 = train['std2']
X_med2 = train['med2']
X_mean2 = train['mean2']

test = get_stats(test,1)
test = get_stats(test,2)
X_test_min1 = test['min1']
X_test_max1 = test['max1']
X_test_std1 = test['std1']
X_test_med1 = test['med1']
X_test_mean1 = test['mean1']
X_test_min2 = test['min2']
X_test_max2 = test['max2']
X_test_std2 = test['std2']
X_test_med2 = test['med2']
X_test_mean2 = test['mean2']

In [6]:
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

#Create the model
#model = simple_cnn()
input_2 = Input(shape=[1], name="angle")
input_3 = Input(shape=[1], name="min1")
input_4 = Input(shape=[1], name="max1")
input_5 = Input(shape=[1], name="std1")
input_6 = Input(shape=[1], name="med1")
input_7 = Input(shape=[1], name="mean1")
input_8 = Input(shape=[1], name="min2")
input_9 = Input(shape=[1], name="max2")
input_10 = Input(shape=[1], name="std2")
input_11 = Input(shape=[1], name="med2")
input_12 = Input(shape=[1], name="mean2")

angle_layer = Dense(1, )(input_2)
min1_layer = Dense(1, )(input_3)
max1_layer = Dense(1, )(input_4)
std1_layer = Dense(1, )(input_5)
med1_layer = Dense(1, )(input_6)
mean1_layer = Dense(1, )(input_7)
min2_layer = Dense(1, )(input_8)
max2_layer = Dense(1, )(input_9)
std2_layer = Dense(1, )(input_10)
med2_layer = Dense(1, )(input_11)
mean2_layer = Dense(1, )(input_12)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(197,197,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = concatenate([x, angle_layer,
                 min1_layer,max1_layer,std1_layer,med1_layer,mean1_layer,
                 min2_layer,max2_layer,std2_layer,med2_layer,mean2_layer])
# x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
# x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[base_model.input,input_2,input_3,input_4,input_5,input_6,
                      input_7,input_8,input_9,input_10,input_11,input_12], 
              outputs=predictions)
#for layer in base_model.layers:
#    layer.trainable = False
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

Using TensorFlow backend.


In [7]:
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam

# 使用不同的优化
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
adagrad = Adagrad(lr = 1e-3, epsilon = 1e-6)
rmsprop = RMSprop(lr=1e-3, rho = 0.9, epsilon=1e-6)
adadelta = Adadelta(lr=1e-3, rho=0.95, epsilon=1e-06)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 16 # 原来是3
#Lets define the image transormations that we want
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.2,
                         rotation_range=10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_twelve_inputs(X1, X2, X3,X4,X5,X6,X7,X8,X9,X10,X11,X12, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,X3, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,X4, batch_size=batch_size,seed=55)
    genX5 = gen.flow(X1,X5, batch_size=batch_size,seed=55)
    genX6 = gen.flow(X1,X6, batch_size=batch_size,seed=55)
    genX7 = gen.flow(X1,X7, batch_size=batch_size,seed=55)
    genX8 = gen.flow(X1,X8, batch_size=batch_size,seed=55)
    genX9 = gen.flow(X1,X9, batch_size=batch_size,seed=55)
    genX10 = gen.flow(X1,X10, batch_size=batch_size,seed=55)
    genX11 = gen.flow(X1,X11, batch_size=batch_size,seed=55)
    genX12 = gen.flow(X1,X12, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            X5i = genX5.next()
            X6i = genX6.next()
            X7i = genX7.next()
            X8i = genX8.next()
            X9i = genX9.next()
            X10i = genX10.next()
            X11i = genX11.next()
            X12i = genX12.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1],X3i[1],X4i[1],X5i[1],X6i[1],X7i[1],
                  X8i[1],X9i[1],X10i[1],X11i[1],X12i[1]], X1i[1]

#Finally create out generator
# gen_flow = gen_flow_for_one_inputs(X_train, y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

'''
epochs_to_wait_for_improve = 10
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint('./model/BestKerasModelResNet50.h5', monitor='val_loss', 
                                      verbose=1, save_best_only=True, mode='min')
'''

#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, 
              X_min1,X_max1,X_std1,X_med1,X_mean1,
              X_min2,X_max2,X_std2,X_med2,X_mean2,
              X_test):
# def myAngleCV(X_train, X_test):
    # K-折交叉验证
    K=4
    
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        X_min1_cv=X_min1[train_idx]
        X_min1_hold=X_min1[test_idx]
        
        X_max1_cv=X_max1[train_idx]
        X_max1_hold=X_max1[test_idx]
        
        X_std1_cv=X_std1[train_idx]
        X_std1_hold=X_std1[test_idx]
        
        X_med1_cv=X_med1[train_idx]
        X_med1_hold=X_med1[test_idx]
        
        X_mean1_cv=X_mean1[train_idx]
        X_mean1_hold=X_mean1[test_idx]
        
        X_min2_cv=X_min2[train_idx]
        X_min2_hold=X_min2[test_idx]
        
        X_max2_cv=X_max2[train_idx]
        X_max2_hold=X_max2[test_idx]
        
        X_std2_cv=X_std2[train_idx]
        X_std2_hold=X_std2[test_idx]
        
        X_med2_cv=X_med2[train_idx]
        X_med2_hold=X_med2[test_idx]
        
        X_mean2_cv=X_mean2[train_idx]
        X_mean2_hold=X_mean2[test_idx]

        #define file path and get callbacks
        file_path = "./model/%s_aug_ResNet_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_twelve_inputs(X_train_cv, X_angle_cv, 
                                              X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                              X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv,
                                              y_train_cv)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         gen_flow = gen_flow_for_one_inputs(X_train_cv,  y_train_cv)
#         galaxyModel= getVggAngleModel()
        galaxyModel= model
    
        # 调整训练参数
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                #steps_per_epoch=100,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold,
                                 X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                 X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                 Y_holdout),
#                 validation_data=(X_holdout, Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv,
                                     X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
                                     X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv], 
                                     y_train_cv, verbose=0)
#         score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
#         score = galaxyModel.evaluate(X_train_cv, y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold,
                                     X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                     X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                     Y_holdout, verbose=0)
#         score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
#         score = galaxyModel.evaluate(X_holdout, Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold,
                                       X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
                                       X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold])
#         pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
#         pred_valid=galaxyModel.predict(X_holdout)
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle,
                                      X_test_min1,X_test_max1,X_test_std1,X_test_med1,X_test_mean1,
                                      X_test_min2,X_test_max2,X_test_std2,X_test_med2,X_test_mean2])
#         temp_test=galaxyModel.predict([X_test, X_test_angle])
#         temp_test=galaxyModel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle,
                                       X_min1,X_max1,X_std1,X_med1,X_mean1,
                                       X_min2,X_max2,X_std2,X_med2,X_mean2])
#         temp_train=galaxyModel.predict([X_train, X_angle])
#         temp_train=galaxyModel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [9]:
preds=myAngleCV(X_train, X_angle, 
                X_min1,X_max1,X_std1,X_med1,X_mean1,
                X_min2,X_max2,X_std2,X_med2,X_mean2,
                X_test)
# preds=myAngleCV(X_train, X_angle, X_test)
# preds=myAngleCV(X_train,X_test)


===================FOLD= 0
Epoch 1/100
24/24 [==============================] - 51s - loss: 1.0268 - acc: 0.5417 - val_loss: 0.6416 - val_acc: 0.5348
Epoch 2/100
24/24 [==============================] - 50s - loss: 0.7133 - acc: 0.6302 - val_loss: 0.5743 - val_acc: 0.6791
Epoch 3/100
24/24 [==============================] - 26s - loss: 0.6849 - acc: 0.6380 - val_loss: 0.5677 - val_acc: 0.6567
Epoch 4/100
24/24 [==============================] - 18s - loss: 0.6236 - acc: 0.6487 - val_loss: 0.5802 - val_acc: 0.6269
Epoch 5/100
24/24 [==============================] - 19s - loss: 0.5950 - acc: 0.6849 - val_loss: 0.5946 - val_acc: 0.7015
Epoch 6/100
24/24 [==============================] - 21s - loss: 0.5488 - acc: 0.7422 - val_loss: 0.6123 - val_acc: 0.5746
Epoch 7/100
24/24 [==============================] - 25s - loss: 0.4989 - acc: 0.7139 - val_loss: 0.5379 - val_acc: 0.7313
Epoch 8/100
24/24 [==============================] - 22s - loss: 0.4572 - acc: 0.7708 - val_loss: 0.5366 - val_

24/24 [==============================] - 21s - loss: 0.1860 - acc: 0.9141 - val_loss: 0.2199 - val_acc: 0.9129
Epoch 67/100
24/24 [==============================] - 19s - loss: 0.2461 - acc: 0.9199 - val_loss: 0.2086 - val_acc: 0.9154
Epoch 68/100
24/24 [==============================] - 21s - loss: 0.2032 - acc: 0.9193 - val_loss: 0.2617 - val_acc: 0.9005
Train loss: 0.170381596352
Train accuracy: 0.936772047085
Test loss: 0.184490011106
Test accuracy: 0.920398012916

===================FOLD= 1
Epoch 1/100
24/24 [==============================] - 28s - loss: 0.2546 - acc: 0.8932 - val_loss: 0.1316 - val_acc: 0.9501
Epoch 2/100
24/24 [==============================] - 22s - loss: 0.2157 - acc: 0.9141 - val_loss: 0.1290 - val_acc: 0.9476
Epoch 3/100
24/24 [==============================] - 20s - loss: 0.2450 - acc: 0.9036 - val_loss: 0.1408 - val_acc: 0.9501
Epoch 4/100
24/24 [==============================] - 19s - loss: 0.2019 - acc: 0.9296 - val_loss: 0.1376 - val_acc: 0.9401
Epoch 5

In [10]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('subTLResNet.csv', index=False)